In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [196]:
from IPython.display import Image
import os
!ls ../input/

# **CNN X-Ray images classification**

Il dataset è organizzato in 3 cartelle (train, test, val) e contiene sottocartelle per ciascuna categoria di immagini (polmonite/normale). Sono disponibili 5.863 immagini a raggi X (JPEG) e 2 categorie (polmonite/normale).

Le immagini radiografiche del torace (anteriore-posteriore) sono state selezionate da studi di coorti retrospettive di pazienti pediatrici di età compresa tra 1 e 5 anni del Guangzhou Women and Children's Medical Center, Guangzhou.

In [197]:
Image("../input/immagini/confronto (1).png")

In [179]:
import os
import cv2
import h5py
import shutil
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

from os import listdir
from os.path import isfile, join
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

from keras import backend as K

In [2]:
train_folder= '/kaggle/input/chest-xray-pneumonia/chest_xray/train/'
val_folder = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/'
test_folder = '/kaggle/input/chest-xray-pneumonia/chest_xray/test/'

In [3]:
os.listdir(train_folder)
train_normal = train_folder+'NORMAL/'
train_pneumonia = train_folder+'PNEUMONIA/'

In [4]:
os.listdir(val_folder)
val_normal = val_folder+'NORMAL/'
val_pneumonia = val_folder+'PNEUMONIA/'

In [5]:
os.listdir(test_folder)
test_normal = test_folder+'NORMAL/'
test_pneumonia = test_folder+'PNEUMONIA/'

In [6]:
labels = ["NORMAL", "PNEUMONIA"] # each folder has two sub folder name "PNEUMONIA", "NORMAL"
IMG_SIZE = 50 # resize image

def get_data_train(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([new_array, class_num])
                img_arr = np.asarray(img_array)
            except Exception as e:
                print(e)
    return np.array(data)

In [29]:
train = get_data_train(train_folder)
test = get_data_train(test_folder)
val = get_data_train(val_folder)

In [8]:
list = []
for i in train:
    if(i[1] == 0):
        list.append("Normal")
    else:
        list.append("Pneumonia")
        
sns.countplot(list)

In [7]:
#Normal pic 
print(len(os.listdir(train_normal)))
#random_normal= np.random.randint(0,len(os.listdir(train_normal)))
normal_picture = os.listdir(train_normal)[1]
print('normal picture title: ',normal_picture)

path_image_normal = train_normal+normal_picture

#Pneumonia
#random_pneumonia = np.random.randint(0,len(os.listdir(train_pneumonia)))
pneumonia_picture =  os.listdir(train_pneumonia)[7]
path_image_pneumonia = train_pneumonia+pneumonia_picture
print('pneumonia picture title:', pneumonia_picture)

# Load the images
normal_load = Image.open(path_image_normal)
pneumonia_load = Image.open(path_image_pneumonia)

#Let's plt these images
f = plt.figure(figsize= (15,10))
a1 = f.add_subplot(1,2,1)
img_plot = plt.imshow(normal_load,cmap='gray', vmin=0, vmax=255)
a1.set_title('Normal')

a2 = f.add_subplot(1, 2, 2)
plt.arrow(450, 500, 200, 300, head_width=30, head_length=30, fc='k', ec='white')
plt.imshow(pneumonia_load,cmap='gray', vmin=0, vmax=255)
a2.set_title('Pneumonia')


# **Building CNN Model**

In [15]:
num_of_test_samples = 500
BATCH_SIZE = 32
height = 180
width = 180
EPOCHS = 25

In [206]:
def build_model():
    model_CNN_Matt = tf.keras.Sequential([
        tf.keras.Input(shape=(64,64,3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same'),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.MaxPool2D(),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Dropout(0.2),
        
        #tf.keras.layers.Dense(512, activation='relu'),
        #tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    return model_CNN_Matt

In [208]:
model_CNN_Matt =  build_model()

In [209]:
model_CNN_Matt.summary()

In [210]:
# Fitting the CNN to the images
# The function ImageDataGenerator augments your image by iterating through image as your CNN is getting ready to process that image

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)  #Image normalization.

training_set = train_datagen.flow_from_directory(train_folder,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(val_folder,
                                                        target_size=(64, 64),
                                                        batch_size=32,
                                                        class_mode='binary')

test_set = test_datagen.flow_from_directory(test_folder,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [211]:
model_CNN_Matt = build_model()
metrics1 = ['accuracy',tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]
model_CNN_Matt.compile(optimizer='adam',loss='binary_crossentropy',metrics=metrics1)

In [212]:
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)

In [213]:
#Fit the model
history = model_CNN_Matt.fit_generator(training_set,
                         epochs = EPOCHS,
                         validation_data = validation_generator)

In [214]:
#evaluation on test set
accuracy = model_CNN_Matt.evaluate(test_set, steps=300)

In [215]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()